# Malta Covid Data Analysys

## Import Libraries

In [151]:
import pandas as pd
import calendar
import plotly.express as px

## Aggregate Cases Dataset

### Load Dataset

In [152]:
cases_df = pd.read_csv('../data/raw/COVID-19 Malta - Aggregate Data Set.csv')

### Group By Month

In [153]:
cases_df['Date'] = pd.to_datetime(cases_df['Date'], format='%d/%m/%Y')
cases_df = cases_df.groupby(cases_df.Date.dt.to_period('M')).agg({
    'New Cases': 'sum',
    'Total Cases': 'last',
    'Recovered': 'last',
    'Deaths': 'last',
    'Active Cases': 'last'
})
cases_df.insert(0, 'Date', cases_df.index.strftime('%m/%Y'))
cases_df.insert(1, 'Year', cases_df.index.year)
cases_df.insert(2, 'Month', cases_df.index.month)
cases_df = cases_df.reset_index(drop=True)
cases_df['Month'] = cases_df['Month'].apply(lambda x: calendar.month_abbr[x])
cases_df = cases_df[cases_df['Year'] != 2023]
cases_df

,Date,Year,Month,New Cases,Total Cases,Recovered,Deaths,Active Cases
0,03/2020,2020,Mar,169,169,2,0,167
1,04/2020,2020,Apr,296,465,351,4,110
2,05/2020,2020,May,153,618,534,9,75
3,06/2020,2020,Jun,52,670,640,9,21
4,07/2020,2020,Jul,154,824,665,9,150
5,08/2020,2020,Aug,1164,1883,1400,12,471
6,09/2020,2020,Sep,1175,3058,2562,34,462
7,10/2020,2020,Oct,2984,6042,4155,62,1825
8,11/2020,2020,Nov,3831,9873,7665,137,2071
9,12/2020,2020,Dec,2901,12774,11121,219,1434


## Total Cases, Recovered & Deaths

In [171]:
fig = px.line(
    cases_df, 
    x='Date', 
    y=['Total Cases', 'Recovered', 'Deaths'], 
    title='Total Cases, Recovered & Deaths'
)
fig.update_layout(
    legend_title='Legend',
    yaxis_title='Cases'
)
fig.show()

### Total Deaths

In [172]:
fig = px.line(cases_df, x='Date', y='Deaths', title='Total Deaths')
fig.show()